In [14]:
# set the matplotlib backend so figures can be saved in the background
#TODO: cleanup, lint
import matplotlib
matplotlib.use("Agg")
 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plot
import librosa
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
    
# import the necessary packages
from musicrec.vgg import VGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras import regularizers
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
from datetime import datetime
import pickle
import cv2
import os
import sys
import tensorflow as tf

from keras.models import Sequential

import warnings
warnings.filterwarnings('ignore')

In [15]:
# Paramters
data_folder = Path("../../../audio/testfiles/GTZAN/genres/")
output_folder = Path("./output/cvnn.model")
spectogram_folder = Path("./img_data/")
# Duration of songsnippet in seconds
duration = 2.97
# Matplotlib colormap for spectogram
spectogram_cmap = 'binary' 
# Predefined list of genres
pred_genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split() 
epochs = 120

# parameters
sr = 22050 # if sampling rate is different, resample it to this

# parameters for calculating spectrogram in mel scale
fmax = 10000 # maximum frequency considered
fft_window_points = 512
fft_window_dur = fft_window_points * 1.0 / sr # 23ms windows
hop_size = int(fft_window_points/ 2) # 50% overlap between consecutive frames
n_mels = 64

# segment duration
num_fft_windows = 256 # num fft windows per music segment
segment_in_points = num_fft_windows * 255 # number of data points that ensure the spectrogram has size: 64 * 256
segment_dur = segment_in_points * 1.0 / sr

num_genres=10
input_shape=(64, 256, 1)

randomseed = 11
#randomseed = datetime.now()
# Seed for RNG
random.seed(datetime.now())

In [16]:
#Get directories of all songs
songs = []
genres = []

spectograms = []

for g in data_folder.iterdir():
    genres.append(g.name)
    for i in g.iterdir():
        songs.append(i)

In [17]:
mel_specs = []  
labels = []
spectograms = []
for song in songs:
    y, sr = librosa.load(song, mono=True, duration=duration)
    m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_window_points,
                                          hop_length=hop_size, n_mels=n_mels,
                                          fmax=fmax)
    #if m_sp.shape != (128, 128): continue
    mel_specs.append(m_sp)
    label = song.parts[-2]
    labels.append(label)
    spectograms.append( (m_sp, label) )
#spectograms = list(zip(mel_specs, labels))
print("Total number of songs: ", len(mel_specs))
input_shape = m_sp.shape + (1,)
input_shape

Total number of songs:  1000


(64, 256, 1)

In [18]:
batch_size = input_shape[1] # use second dim as batch size, as it varies with duration

In [19]:
imagesize_x = np.shape(mel_specs)[1]
imagesize_y = np.shape(mel_specs)[2]
print("Spectogram dimensions: ", imagesize_x, "x", imagesize_y)

Spectogram dimensions:  64 x 256


In [20]:
# Shuffle the spectograms
random.shuffle(spectograms)
#spectograms[0][1]

In [21]:
#TODO: change
testsplit = len(mel_specs)*2/3
train = spectograms[:int(testsplit)]
test = spectograms[int(testsplit):]

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape(input_shape) for x in X_train])
X_test = np.array([x.reshape(input_shape) for x in X_test])

# One-Hot encoding for classes
lb = LabelBinarizer()
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.transform(y_test)


"""
label_encoder = LabelEncoder()
y_train_int = label_encoder.fit_transform(y_train)
y_test_int = label_encoder.fit_transform(y_test)
#print(integer_encoded)

y_train_bin = np.array(keras.utils.to_categorical(y_train_int, 10))
y_test_bin = np.array(keras.utils.to_categorical(y_test_int, 10))
#print("Encoding of test data:\n", y_test, "=>\n", y_test_int, "=>\n", y_test_bin)
"""
0

0

In [22]:
y_train = y_train_bin
y_test = y_test_bin

In [23]:
"""

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2,horizontal_flip=True, 
                         fill_mode="nearest")
 
# initialize our VGG-like Convolutional Neural Network
model = VGGNet.build(width=imagesize_x, height=imagesize_y, depth=1, 
                          classes=len(lb.classes_))
"""
0

0

In [24]:
#old model for urban
"""
model = Sequential()

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))
"""
#model for GTZAN


model = Sequential()
model.add(Conv2D(96, kernel_size=(3, 3),
                 activation='relu', kernel_regularizer=regularizers.l2(0.01),
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 4)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 5), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(pool_size=(2, 4)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
model.add(Dropout(0.2))
model.add(Dense(num_genres, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(decay=1e-5),
              metrics=['accuracy'])

In [25]:
#model.compile(
#	optimizer="Adam",
#	loss="categorical_crossentropy",
#	metrics=['accuracy'])

H = model.fit(
	x=X_train, 
	y=y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data= (X_test, y_test))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 666 samples, validate on 334 samples
Epoch 1/120
666/666 [==============================] - 48s 72ms/step - loss: 4.5699 - acc: 0.1081 - val_loss: 4.1273 - val_acc: 0.1228
Epoch 2/120
666/666 [==============================] - 45s 68ms/step - loss: 4.0393 - acc: 0.1306 - val_loss: 3.8941 - val_acc: 0.1168
Epoch 3/120
666/666 [==============================] - 38s 57ms/step - loss: 3.8136 - acc: 0.2087 - val_loss: 3.7660 - val_acc: 0.1257
Epoch 4/120
666/666 [==============================] - 39s 58ms/step - loss: 3.6098 - acc: 0.1697 - val_loss: 3.6196 - val_acc: 0.1737
Epoch 5/120
666/666 [==============================] - 38s 57ms/step - loss: 3.4293 - acc: 0.2312 - val_loss: 3.5472 - val_acc: 0.1527
Epoch 6/120
666/666 [==============================] - 40s 60ms/step - loss: 3.3071 - acc: 0.2477 - val_loss: 3.3586 - val_acc: 0.2036
Epoch 7/120
666/666 [==============================] - 42s 63ms/step - loss: 3.1627 - acc: 0.2673 - val_loss: 3.2487 - val_acc: 0.1946
Epoch 8/1

Epoch 61/120
666/666 [==============================] - 49s 74ms/step - loss: 0.8854 - acc: 0.8724 - val_loss: 3.6306 - val_acc: 0.3683
Epoch 62/120
666/666 [==============================] - 49s 74ms/step - loss: 0.8725 - acc: 0.8784 - val_loss: 3.7604 - val_acc: 0.3353
Epoch 63/120
666/666 [==============================] - 49s 74ms/step - loss: 0.8858 - acc: 0.8724 - val_loss: 3.9915 - val_acc: 0.3323
Epoch 64/120
666/666 [==============================] - 49s 74ms/step - loss: 0.9094 - acc: 0.8634 - val_loss: 4.1443 - val_acc: 0.3114
Epoch 65/120
666/666 [==============================] - 49s 74ms/step - loss: 1.0425 - acc: 0.8018 - val_loss: 3.8076 - val_acc: 0.3623
Epoch 66/120
666/666 [==============================] - 51s 76ms/step - loss: 0.8917 - acc: 0.8874 - val_loss: 3.7924 - val_acc: 0.3503
Epoch 67/120
666/666 [==============================] - 50s 76ms/step - loss: 0.8378 - acc: 0.8994 - val_loss: 3.9006 - val_acc: 0.3623
Epoch 68/120
666/666 [==========================

KeyboardInterrupt: 

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(X_test, batch_size=batch_size)
print(classification_report(y_test_bin.argmax(axis=1),
                            predictions.argmax(axis=1), 
                            target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
"""
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
"""
0